In [ ]:
#Test

%%Compulsory task scenario C

function [S,V1,V2,V3,L1,L2,L3,xL1,xL2,A,TV1,TV2]=Compulsory_Task % Requires
% HSteam
% hLBlackLiquor

[F,xF,xLout,Tf,Ts,T]=known();
X=[7 7 7 7 7 7 7 0.375 0.375 200 90 90]; %Set initial guesses options = optimoptions(@fsolve,'MaxIter',1000,'TypicalX',X');

Res=fsolve(@evaporator,X,options);

% Translate results to variable names easier to understand

S=Res(1); V1=Res(2); V2=Res(3); V3=Res(4); L1=Res(5); L2=Res(6); L3=Res(7); xL1=Res(8); xL2=Res(9); A=Res(10); TV1=Res(11); TV2=Res(12);

% You can use the command "fprintf" to print results
% (but note that you can also copy entire matrices of results e.g. to Word) fprintf('%12s: %d %4s \n','Steam flux',S,'kg/s' )
fprintf('%12s: %d %4s \n','Vapor flux 1',V1,'kg/s' )
fprintf('%12s: %d %4s \n','Vapor flux 2',V2,'kg/s' )
fprintf('%12s: %d %4s \n','Vapor flux 3',V3,'kg/s' )

14

fprintf('%12s: %d %4s \n','Liquid flux 1',L1,'kg/s' ) fprintf('%12s: %d %4s \n','Liquid flux 2',L2,'kg/s' ) fprintf('%12s: %d %4s \n','Liquid flux 3',L3,'kg/s' ) fprintf('%12s: %d %4s \n','Product composition 1',xL1,'kg/kg' ) fprintf('%12s: %d %4s \n','Product composition 2',xL2,'kg/kg' ) fprintf('%12s: %d %4s \n','Area',A,'m2' )

fprintf('%12s: %d %4s \n','Vapour temperature 1',TV1,'Â°C' ) fprintf('%12s: %d %4s \n','Vapour temperature 2',TV2,'Â°C' )

[Hs,P]=HSteam(Ts,-1) [HV,P]=HSteam(T,-1)

end
function [F,xF,xLout,Tf,Ts,T]=known()

% =========== Hard coded, problem-specific values ==========

F=15;% kg/s Feed flux
xF=0.15; % kg dry matter/kg total Feed dry matter content
xLout=0.6;% kg dry matter/kg total Dry matter content from last evaporator Tf=40; % degrees C Feed temperature
Ts=158.83; %degrees C
T=45.81; % degrees C Temperature for vapuor flow from last evaporator

end
function Y=evaporator(X)

%Calculates residuals of mass and energy balances for an evaporator

% ================ Initialisation ========================= %
% Copy X-vector to variables that are easier to understand
S=X(1);

V1=X(2); V2=X(3); V3=X(4); L1=X(5); 

CT2

L2=X(6); L3=X(7); xL1=X(8); xL2=X(9); A=X(10); TV1=X(11); TV2=X(12);

% Fetch known parameters using function "known"

[F,xF,xLout,Tf,Ts,T]=known();

% =============== Calculations start here ================== % Calculation of water vapor enthalpies

Hs=HSteam(Ts,-1);

beta1=BPE(xL1); beta2=BPE(xL2); beta3=BPE(xLout);

HV1=HSteam(TV1-beta1,TV1); HV2=HSteam(TV2-beta2,TV2); HV3=HSteam(T,T+beta3);

hL1=hLBlackLiquor(xL1,TV1); hL2=hLBlackLiquor(xL2,TV2); hL3=hLBlackLiquor(xLout,T+beta3);

hk1=hLBlackLiquor(0,Ts); hk2=hLBlackLiquor(0,TV1-beta1); hk3=hLBlackLiquor(0,TV2-beta2);

hf=hLBlackLiquor(xF,Tf);

k1=kBlackLiquor(xF,xL1,TV1,F,L1); k2=kBlackLiquor(xL1,xL2,TV2,L1,L2); k3=kBlackLiquor(xL2,xLout,(T+beta3),L2,L3);

% ================ Calculating residuals: ================== %
%

16

% evaporator 1
Y(1)=V1+L1-F; % MB total
Y(2)=F*xF-L1*xL1; % MB solids Y(3)=S*(Hs-hk1)+F*hf-V1*HV1-L1*hL1; % EB over evaporator Y(4)=S*(Hs-hk1)-k1*A*(Ts-TV1); % EB over heat exchanger

%evaporator 2
Y(5)=V2+L2-L1; % MB total
Y(6)=L2*xL2-L1*xL1; % MB solids Y(7)=V1*(HV1-hk2)-V2*HV2-L2*hL2+L1*hL1; % EB over evaporator Y(8)=V1*(HV1-hk2)-k2*A*(TV1-TV2-beta1); % EB over heat exchanger

%evaporator 3
Y(9)=V3+L3-L2; % MB total
Y(10)=L3*xLout-L2*xL2; % MB solids Y(11)=V2*(HV2-hk3)-V3*HV3-L3*hL3+L2*hL2; % EB over evaporator Y(12)=V2*(HV2-hk3)-k3*A*(TV2-(T+beta3)-beta2); % EB over heat exchanger 